## 初始準備

In [32]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [33]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

## 讀入 IMDB 電影數據庫

In [34]:
from keras.datasets import imdb

In [35]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

要注意這裡我們限制只選「最常用」1 萬字, 也就是超過這範圍的就當不存在。這是文字分析常會做的事。

In [36]:
print('訓練總筆數:', len(x_train))
print('測試總筆數:', len(x_test))

訓練總筆數: 25000
測試總筆數: 25000


### 送入神經網路的輸入處理

雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

* 設輸入文字長度的上限
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [37]:
from keras.preprocessing import sequence

In [38]:
x_train = sequence.pad_sequences(x_train, maxlen=150)
x_test = sequence.pad_sequences(x_test, maxlen=150)

In [39]:
x_train.shape

(25000, 150)

## 以LSTM打造RNN

### 決定神經網路架構

* 先將 10000 維的文字壓到 N 維
* 然後用 K 個 LSTM 神經元做隱藏層
* 最後一個 output, 直接用 sigmoid 送出

### 建構神經網路

文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少 (10000), 還有我們打算壓到幾維 (N)。

In [40]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import Dropout

In [63]:
N = 530 # 文字要壓到 N 維
K = 20 # LSTM 有 K 個神經元

In [64]:
model = Sequential()

In [65]:
model.add(Embedding(10000, N))

LSTM 層, 我們做 K 個 LSTM Cells。

In [66]:
model.add(Dropout(0.2))
model.add(LSTM(K))
model.add(Dropout(0.4))

單純透過 sigmoid 輸出。

In [67]:
model.add(Dense(1, activation='sigmoid'))

### 組裝

這次我們用 binary_crossentropy 做我們的 loss function, 另外用一個很潮的 Adam 學習法。

In [68]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [69]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 530)         5300000   
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 530)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 20)                44080     
_________________________________________________________________
dropout_6 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 21        
Total params: 5,344,101
Trainable params: 5,344,101
Non-trainable params: 0
_________________________________________________________________


## 訓練

In [70]:
model.fit(x_train, y_train,
         batch_size=24,
         epochs=2)

Epoch 1/2
25000/25000 [==============================] - 231s 9ms/step - loss: 0.4271 - acc: 0.8038
Epoch 2/2
25000/25000 [==============================] - 229s 9ms/step - loss: 0.2585 - acc: 0.8978


## 檢視結果

### 分數

我們照例來看看測試資料的分數。

In [71]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 32s 1ms/step


In [72]:
print(f'測試資料的 loss = {score[0]}')
print(f'測試資正確率 = {score[1]}')

測試資料的 loss = 0.307587512011528
測試資正確率 = 0.871


### 儲存結果



In [73]:
model_json = model.to_json()
open('imdb_model_arch.json',
     'w').write(model_json)
model.save_weights('imdb_model_weights.h5')